Установка зависимостей

In [ ]:
!pip install torch torchvision matplotlib

Импорты и подготовка данных

In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

Создание нейросети

In [ ]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(28*28, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 10)

  def forward(self, x):
    x = x.view(-1, 28*28)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    return self.fc3(x)

model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Обучение модели

In [ ]:
for epoch in range(5):
  running_loss = 0
  correct = 0
  total = 0

  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item() * images.size(0)
    _, predicted = torch.max(outputs, 1)
    correct += (predicted == labels).sum().item()
    total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_accuracy = correct / total * 100

    print(f'Epoch {epoch+1} — Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

Тестирование

In [ ]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        accuracy = correct / total * 100

print(f'Accuracy on test set: {accuracy:.2f}%')


Визуализация

In [ ]:
images, labels = next(iter(test_loader))
image = images[0]
label = labels[0]

with torch.no_grad():
    output = model(image.to(device).unsqueeze(0))
    _, predicted = torch.max(output, 1)

plt.imshow(image.squeeze(), cmap='gray')
plt.title(f'Predicted: {predicted.item()}, True: {label}')
plt.axis('off')
plt.show()